In [1]:
import importlib
import test_extract
import graph
importlib.reload(test_extract)
importlib.reload(graph)
import time
import matplotlib.pyplot as plt

In [17]:
start = time.time()
data = test_extract.Data("./data/CIS_enrollment.csv", ["Computer Science"])
end = time.time()
print(end - start)

0.9292340278625488


# Data model test

In [18]:
len(data.courses),len(data.students)

(2946, 3194)

In [19]:
print(list(data.students.values())[0])

{'id': '321398', 'major': 'Computer Science', 'courses': ['MicroElectro Mechanical System', 'Multivariable Calculus Engrs', 'Computer Graphics I', 'Fiber & Integrated Optics', 'Intro to Scientific Comp', 'Lasers and Optoelectronics', 'Datacenter Networks & Services', 'Intro Microeconomics', 'Advanced Volleyball', 'Differential Equations Engrs', 'Digital VLSI Design', 'Embedded Systems', 'Linear Algebra for Engineers', 'Physics II: Electromagnetism', 'Engineering Communications', 'Marketing', 'Intro to Microelectronics', 'Advanced Digital VLSI', 'Intro Computing Using Java', 'Advanced Systems', 'Intro to Entrep & Engr Enterpr', 'Archery', 'Beginning Swim', 'Advanced Computer Architecture', 'Phys III-Osc Waves & Quan Phys', 'Short Course in MATLAB', 'Intro to Circ Elec & Comp Engr', 'Advanced Computer Architecture', 'Physics I: Mechanics & Heat', 'Cooperative Workshop MATH 1920', 'Signals and Information', 'Intro to Business Management', 'Finance', 'Computer Architecture', 'Digital Logic

In [20]:
print(list(data.courses["CS1110"].students)[0])

{'id': '2995821', 'major': 'Computer Science', 'courses': ['Introductory Oceanography', 'Systems Programming', 'Intro Computing Using Java', 'Foundations of Artif Inllgnce', 'Discrete Structures', 'Intro Analysis of Algorithms', 'Introduction To Linguistics', 'History of the Mod Mideast', 'Obj-Oriented Prog & Data Struc', 'Linear Algebra for Engineers', 'Financial Accounting', 'Beginning Swim', 'Managerial Finance', 'Intro Microeconomics', 'Intro Bio: Comp Physiology', 'Introduction to Psychology', 'Data Struct & Functional Progr', 'Probability Models & Inference', 'Machine Learning'], 'terms': [('PE1100', (2016, 2)), ('PSYCH1101', (2018, 2)), ('CS3110', (2018, 0)), ('CS4780', (2018, 2)), ('CS4700', (2018, 0)), ('MATH2940', (2017, 2)), ('CS2110', (2017, 0)), ('NES2674', (2017, 2)), ('ECON1110', (2017, 2)), ('CS1110', (2016, 2)), ('BIOEE1540', (2018, 2)), ('LING1101', (2018, 0)), ('BIOG1440', (2016, 2)), ('CS3410', (2017, 2)), ('BTRY3080', (2018, 0)), ('CS2800', (2017, 0)), ('AEM2210', 

# Graph model test

In [28]:
g = graph.Graph()

In [29]:
start = time.time()
c = 1
for cname1 in data.courses:
    if c % 250 == 0:
        print(c)
    for cname2 in data.courses:
        if cname2 == cname1:
            continue
        course = data.courses[cname1]
        count = 0
        for s in course.students:
            if cname2 in s.term_numbers and s.term_numbers[cname2] == s.term_numbers[cname1] + 2:
                count += 1
        if count > 0:
            g.addEdge(cname1, cname2, count)
    c += 1
end = time.time()
print(end - start)

250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
43.71582221984863


In [30]:
len(g.edges)

96242

In [31]:
for e in g.filterEdges(lambda e: e.weight > 250):
    print(str(e))

(MATH2940,CS3110,371)
(MATH2940,CS4410,329)
(MATH2940,CS3410,308)
(MATH2940,CS4820,388)
(PHYS1112,CS2800,266)
(ENGRG1050,MATH2940,714)
(ENGRG1050,CS2800,688)
(ENGRG1050,MATH2930,300)
(ENGRG1050,CS3110,544)
(ENGRG1050,PHYS2213,576)
(ENGRG1050,ENGRD2700,362)
(ENGRG1050,CS2850,398)
(MATH1920,MATH2940,526)
(MATH1920,CS2800,513)
(MATH1920,MATH2930,296)
(MATH1920,CS3110,522)
(MATH1920,PHYS2213,287)
(MATH1920,ENGRD2700,351)
(MATH1920,CS3410,379)
(MATH1920,CS2850,294)
(CS1110,MATH2940,368)
(CS1110,CS2800,531)
(CS1110,CS3110,442)
(CS1110,PHYS2213,288)
(CS1110,CS3410,261)
(CHEM2090,MATH2940,312)
(CHEM2090,CS2800,298)
(CHEM2090,PHYS2213,301)
(CS2800,CS3110,564)
(CS2800,CS4410,443)
(CS2800,CS4701,280)
(CS2800,CS3410,496)
(CS2800,CS4820,586)
(CS2800,CS4700,356)
(MATH1910,MATH2940,338)
(MATH1910,CS2800,263)
(MATH1910,PHYS2213,305)
(CS3110,CS4410,641)
(CS3110,CS4701,310)
(CS3110,CS4780,279)
(CS3110,CS4820,383)
(CS3110,CS4700,323)
(PHYS2213,CS3110,265)
(CS2110,CS3110,402)
(CS2110,CS3410,431)
(CS2110,C

In [32]:
g.export_graph("test4",300)

done! exported as test4.dot


In [36]:
x = [min(e.weight, 100) for e in g.edges]
num_bins = 25
n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)
plt.xlabel("edge weight")
plt.ylabel("frequency")
plt.title("distribution of edge weights (capped at 100)")
plt.show()

AttributeError: 'tuple' object has no attribute 'weight'

In [121]:
len(g.edges)

159731

In [134]:
edges_dict = {}
for e in g.edges:
    edges_dict[(e.src, e.dest)] = e.weight
print(len(edges_dict))

159731


In [135]:
for k in list(edges_dict.keys()):
    k1, k2 = k
    if k in edges_dict and (k2,k1) in edges_dict:
        w1, w2 = edges_dict[k], edges_dict[(k2, k1)]
        if w1/w2 > 4:
            del edges_dict[(k2, k1)]
        elif w2/w1 > 4:
            del edges_dict[k]
print(len(edges_dict))

156744


In [139]:
for k in list(edges_dict.keys()):
    if edges_dict[k] < 5:
        del edges_dict[k]
print(len(edges_dict))    

14225


In [141]:
n = set()
for k in list(edges_dict.keys()):
    n.add(k[0])
    n.add(k[1])
print(len(n))

922


In [142]:
len(g.nodes)

3625

In [146]:
def export_edges(edges, name:str):
    nodes = set([e.src for e in edges]).intersection(set([e.dest for e in edges]))
    fname = name+".dot"
    with open(fname, "w") as f:
        f.write("digraph course_graph {\n")
        for e in edges:
            f.write('{} -> {} [ label = "{}" ];\n'.format(e.src, e.dest, e.weight))
        for n in nodes:
            f.write('{} [ label = "{}" ];\n'.format(n,n))
        f.write("}\n")
    print("done! exported as {}.dot".format(name))

In [149]:
new_edges = [e for e in g.edges if (e.src, e.dest) in edges_dict and e.weight > 250]
export_edges(new_edges, "test2")

done! exported as test2.dot
